-------
<h1 align="center"> <font size="30"> CS5785 HW 2 -- Applied Machine Learning </font>  </h1>  

-------

<h1 align="center"> <font size="5"> Student: Shangjing Tang(st787)， Katherine Wu(Kw634) </font> </h1>  
&nbsp;
&NewLine;
&NewLine;

<h1 align="center"> <font size="5"> Due Date: September 29th </font> </h1> 


<div style="page-break-after: always; break-after: page;"></div>
<h1 align="center"> <font size="5"> PROGRAMMING EXERCISES </font> </h1>

#### Part I.  Binary Classification on Text Data
In this problem,you will implement several machine learning techniques from the class to perform classification on text data. Throughout the problem, we will be working on the NLP with Disaster Tweets Kaggle competition, where the task is to predict whether or not a tweet is about a real disaster.

* Download the training and test data from Kaggle,and answer the follow-ing questions: (1) how many training and test data points are there? and (2) what percentage of the training tweets are of real disasters, and what percentage is not? Note that the meaning of each column is explained in the data description on Kaggle.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import os
import warnings
import seaborn as sns
import string
from scipy.stats import skew
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn import datasets
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, make_scorer



pd.options.mode.chained_assignment = None  # default='warn'
warnings.filterwarnings('ignore')
%matplotlib inline

workingDirectory = os.getcwd()
trainData = pd.read_csv(workingDirectory+"/train.csv")
testData = pd.read_csv(workingDirectory+ "/test.csv")
all_data = pd.concat([trainData, testData], axis=0) #merge both trainig

train_target = pd.DataFrame(trainData['target'].value_counts()) 

print("The number of training data points are:", len(trainData.index))
print("The number of testing data points are:", len(testData.index))
print("The percentage of real disaster in trainning data is %.2f%%" %(train_target.loc[1]/len(trainData.index) *100)  )
print("The percentage of fake disaster in trainning data is %.2f%%" % (train_target.loc[0]/len(trainData.index)*100))

# plt.style.use('seaborn-whitegrid')
# plt.hist(con_feat1, bins=30,facecolor = '#2ab0ff', edgecolor='#169acf', linewidth=0.5)  # density=False would make counts
# plt.ylabel('distribution')
# plt.xlabel('Sale price')

# plt.show()

# plt.style.use('seaborn-whitegrid')
# plt.hist(cat_feat4, bins=30,facecolor = '#2ab0ff', edgecolor='#169acf', linewidth=0.5)  # density=False would make counts
# plt.ylabel('distribution')
# plt.xlabel('sale condition')

# plt.show()





The number of training data points are: 7613
The number of testing data points are: 3263
The percentage of real disaster in trainning data is 42.97%
The percentage of fake disaster in trainning data is 57.03%


In [2]:
len(all_data)

10876

* Since the data consists of tweets,they may contain significant amounts of noise and unprocessed content. You may or may not want to do one or all of the following. Explain the reasons for each of your decision (why or why not).
    - Convert all the words to lower case.
    - Lemmatize all the words (i.e.,convert every word to its root so that all of “running,”“run,” and “runs” are converted to “run” and and all of “good,” “well,” “better,” and “best” are converted to “good”; this is easily done using nltk.stem).
    - Strip punctuation.
    - Strip the stop words,e.g.,“the”,“and”,“or”.
    - Strip @ and urls.(It’sTwitter.)
    - Something else? Tell us about it.

In [263]:
import nltk
import re
import spacy

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prettygirl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/prettygirl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/prettygirl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/prettygirl/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [264]:
## Preprocessing -- Lowercase----------------------------------------------------------------------------------------------------------
trainData['text'] = trainData['text'].str.lower()



## strip urls-------------------------------------------------------------------------------------------------------------------
for l in range(0,len(trainData['text'])): 
    trainData['text'][l] = re.sub(r'http\S+', '', trainData['text'][l])
    trainData['text'][l] = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', trainData['text'][l])

    
## get ride of punctuation----------------------------------------------------------------------------------------------------------
trainData['text'] = trainData['text'].str.translate(str.maketrans('', '', string.punctuation))



## strip stop words ----------------------------------------------------------------------------------------------------------
text_token = []
temp = trainData['text']
for i in range(0,len(trainData['text'])): 
   text_token.append(word_tokenize(temp[i]))
for row in range(0,len(text_token)):
    for word in text_token[row]:
        if word in stopwords.words('english'):
            text_token[row].remove(word)
        
            
    text_token[row] =  (" ").join(text_token[row])        


In [265]:

## Lamatize ----------------------------------------------------------------------------------------------------------------

nlp = spacy.load('en_core_web_sm')

clean_token = []
lemmatized_sentence = []

for l in range(0,len(text_token)):
    text_token[l] = nlp(text_token[l])
    for tokes in text_token[l]:
        clean_token.append(tokes)
        
    lemmatized_sentence.append(" ".join([tokes.lemma_ for tokes in text_token[l]]))  
    

clean_sentence = pd.DataFrame({'clean_Text':lemmatized_sentence})
new_trainData = pd.concat([trainData, clean_sentence],join = 'outer', axis = 1)
   

In [247]:
##sanity check
new_trainData

,id,keyword,location,text,target,clean_Text
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1,deed the reason this earthquake may allah forg...
1,4,NaN,NaN,forest fire near la ronge sask canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,all residents asked to shelter in place are be...,1,resident ask shelter place be notify officer o...
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1,13000 people receive wildfire evacuation order...
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1,got send photo ruby alaska smoke wildfire pour...
...,...,...,...,...,...,...
7608,10869,NaN,NaN,two giant cranes holding a bridge collapse int...,1,two giant crane hold bridge collapse nearby home
7609,10870,NaN,NaN,ariaahrary thetawniest the out of control wild...,1,ariaahrary thetawni out control wild fire cali...
7610,10871,NaN,NaN,m194 0104 utc5km s of volcano hawaii,1,m194 0104 utc5 km of volcano hawaii
7611,10872,NaN,NaN,police investigating after an ebike collided w...,1,police investigate an ebike collide a car litt...


<span style="color:blue"> 
    During the preprocessing step, we decided to first convert all letters into lowercase, becasue we don't want the model to treat same contents differently based on their cases. We then stripped all the urls from the text by using re.sub becasue urls are not helpful information in our cases. After this we strip other useless information such as punctuations and stopwords. Finally, we perform lamatization to each word token to rind their wording root. 

* Split the training data. Since we do not know the correct values of labels in the test data, we will split the training data from Kaggle into a training set and a development set (a de- velopment set is a held out subset of the labeled data that we set aside in order to fine-tune models, before evaluating the best model(s) on the test data). Randomly choose 70% of the data points in the training data as the training set, and the remaining 30% of the data as the development set. Throughout the rest of this problem we will keep these two sets fixed. The idea is that we will train different models on the training set, and compare their performance on the development set, in order to decide what to submit to Kaggle.

In [248]:
newData = pd.concat([trainData['target'], clean_sentence],join = 'outer', axis = 1)
y = newData['target']
indep = newData['clean_Text']
x_train, x_valid,y_train,y_valid = train_test_split(indep, y, test_size = 0.3, random_state = 0)


In [249]:
##sanity check
x_train.head()

476     blazerfan everyone see ignoranceshe latinoand ...
4854    white people know worry tirelessly black black...
4270                  chilli heat wave doritos never fail
992     broseidonrex dapurplesharpie skim twitter miss...
4475    hot c130 specially modify land a stadium rescu...
Name: clean_Text, dtype: object

In [250]:
##sanity check
y_valid.head()

311     0
4970    0
527     0
6362    0
800     0
Name: target, dtype: int64

<span style="color:blue"> We split the data and randomly choose 70% of data as training data. Please see the sanity check above. .</span>

* Bag of words model.The next task is to extract features in order to represent each tweet using the binary “bag of words” model, as discussed in lectures. The idea is to build a vocabulary of the words appearing in the dataset, and then to represent each tweet by a feature vector x whose length is the same as the size of the vocabulary, where xi = 1 if the i ’th vocabulary word appears in that tweet, and xi = 0 otherwise. In order to build the vocabulary, you should choose some threshold M, and only include words that appear in at least k different tweets; this is important both to avoid run-time and memory issues, and to avoid noisy/unreliable features that can hurt learning. Decide on an appropriate threshold M , and discuss how you made this decision. Then, build the bag of words feature vectors for both the training and development sets, and report the total number of features in these vectors. 
    
    In order to construct these features, we suggest using the CountVectorizer class in sklearn. A couple of notes on using this function: (1) you should set the option “binary=True” in order to ensure that the feature vectors are binary; and (2) you can use the option “min_df=M” in or- der to only include in the vocabulary words that appear in at least M different tweets. Finally, make sure you fit CountVectorizer only once on your training set and use the same instance to process both your training and development sets (don’t refit it on your development set a second time).

    Important: at this point you should only be constructing feature vectors for each data point using the text in the “text” column. You should ignore the “keyword” and “location” columns for now.

In [145]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
for i in range(1,9):
    CountVec = CountVectorizer(binary = True, min_df = i)
    #transform
    CountVec_train = CountVec.fit_transform(x_train).toarray()
    CountVec_valid = CountVec.transform(x_valid).toarray()
    logreg = LogisticRegression()
    logreg.fit(CountVec_train,y_train)
    predict_y = logreg.predict(CountVec_valid)
    print('With M = %s, F1 score = %.4f' % (i, f1_score(y_valid, predict_y)))

With M = 1, F1 score = 0.7475
With M = 2, F1 score = 0.7508
With M = 3, F1 score = 0.7425
With M = 4, F1 score = 0.7403
With M = 5, F1 score = 0.7392
With M = 6, F1 score = 0.7348
With M = 7, F1 score = 0.7324
With M = 8, F1 score = 0.7303


In [157]:
## use M = 2
count_vect_real = CountVectorizer(binary=True, min_df=2)
CountVec_train_real = count_vect_real.fit_transform(x_train).toarray()
vnames = count_vect_real.get_feature_names()
print(vnames)
CountVec_valid_real = count_vect_real.transform(x_valid).toarray()
CountVec_valid_real.shape

['001116', '0104', '010401', '02', '0306', '05', '06', '070', '0700', '075', '0800', '09', '0day', '10', '100', '1000', '100000', '101', '1030', '1030pm', '10401', '1061thetwister', '109', '10th', '11', '1100', '118', '11yearold', '12', '1200', '12000', '125', '13', '13000', '133', '14', '15', '150', '150401', '150foot', '15th', '16', '161116', '1620', '166', '16yr', '17', '1716', '18', '180', '1800', '18w', '18wheeler', '19', '1916', '1943', '1965', '1976', '1979', '1980', '1998', '1st', '20', '2005', '2008', '2010', '2011', '2013', '2014', '2015', '20150805', '201516', '2030', '2082676773', '21', '21a', '22', '23', '233liveonline', '235409', '24', '25', '26', '27', '29', '29072015', '2nd', '2pcs', '2us', '30', '300', '300000', '300w', '31', '32', '320', '33', '34', '35', '36', '360', '360wisenews', '361', '370', '3942', '3d', '3rd', '40', '400', '4000', '45', '48', '4playthursdays', '4th', '4wd', '4x4', '50', '500', '5000', '53', '530', '53inch', '55', '56', '582', '5c', '5s', '5sos'

(2284, 4429)

* Logistic regression. In this question, we will be training logistic regression models using bag of words feature vectors obtained in part (d). We will use the F 1-score as the evaluation metric.

    We use F 1-score because it gives a more comprehensive view of classifier performance than
    accuracy. For more information on this metric see F1-score.
    We ask you to train the following classifiers. We suggest using the LogisticRegression implementation in sklearn.
    
    * Train a logistic regression model without regularization terms. You will notice that the default sklearn logistic regression utilizes L2 regularization. You can turn off L2 regu- larization by changing the penalty parameter. Report the F1 score in your training and in your development set. Comment on whether you observe any issues with overfitting or underfitting.
    * Train a logistic regression model with L1 regularization. Sklearn provides some good examples for implementation. Report the performance on both the training and the development sets.
    * Similarly, train a logistic regression model with L2 regularization. Report the perfor- mance on the training and the development sets.
    * Which one of the three classifiers performed the best on your training and development set? Did you observe any overfitting and did regularization help reduce it? Support your answers with the classifier performance you got.
    * Inspect the weight vector of the classifier with L1 regularization (in other words, look at the θ you got after training). You can access the weight vector of the trained model using the coef_ attribute of a LogisticRegression instance. What are the most important words for deciding whether a tweet is about a real disaster or not?
    

In [226]:
#logistics regression without regulation 
lr = LogisticRegression(penalty = 'none', C = 9999)
lr.fit (CountVec_train_real, y_train)
prediction_y = lr.predict(CountVec_valid_real)
print('F1 score with logreg: %.4f' % f1_score(y_valid, prediction_y))
weights = lr.coef_[0]
sorted_weight_index = np.argsort(np.abs(lr.coef_[0])) #return the index of the sorted weights 
for i in sorted_weight_index[-10:]:
    print(vnames[i])


F1 score with logreg: 0.7079
hurt
outbreak
mayhem
plane
source
40
myanmar
hiroshima
typhoon
bombing


In [224]:
#logistics regression with regulation L2
lr_l2 = LogisticRegression(penalty = 'l2')
lr_l2.fit (CountVec_train_real, y_train)
prediction_y = lr_l2.predict(CountVec_valid_real)
print('F1 score with logreg: %.4f' % f1_score(y_valid, prediction_y))
weights = lr_l2.coef_[0]
sorted_weight_index = np.argsort(np.abs(lr_l2.coef_[0])) #return the index of the sorted weights 
for i in sorted_weight_index[-10:]:
    print(vnames[i])

F1 score with logreg: 0.7508
california
drought
storm
evacuate
earthquake
typhoon
wildfire
massacre
bombing
hiroshima


In [225]:
#logistics regression with regulation L1
lr_l1 = LogisticRegression(C=1, penalty='l1', solver='liblinear')
lr_l1.fit (CountVec_train_real, y_train)
prediction_y = lr_l1.predict(CountVec_valid_real)
print('F1 score with logreg: %.4f' % f1_score(y_valid, prediction_y))
weights = lr_l1.coef_[0]
sorted_weight_index = np.argsort(np.abs(lr_l1.coef_[0])) #return the index of the sorted weights 
for i in sorted_weight_index[-10:]:
    print(vnames[i])

F1 score with logreg: 0.7412
myanmar
derailment
massacre
outbreak
earthquake
mh370
migrant
hiroshima
bombing
typhoon


In [200]:
from sklearn.model_selection import cross_validate, cross_val_score, KFold

#cross validation for logistics regression without regulation 
cv_dict = cross_validate(lr, CountVec_train_real, y_train, return_train_score=True)
kfolds = KFold(n_splits = 10, random_state = 7)
cv_dictKfold = cross_val_score(lr,CountVec_train_real,y_train,cv = kfolds, scoring ='accuracy')
print('The cross validation result for logistics regression without regulation is ', cv_dict)
#cross validation for logistics regression with l2 regulation
cv_dict_l2 = cross_validate(lr_l2, CountVec_train_real, y_train, return_train_score=True)
cv_dictKfold_l2 = cross_val_score(lr_l2,CountVec_train_real,y_train,cv = kfolds, scoring ='accuracy')

print('The cross validation result for logistics regression with l2 regulation is ', cv_dict_l2)

#cross validation for logistics regression with l1 regulation 
cv_dict_l1 = cross_validate(lr_l1, CountVec_train_real, y_train, return_train_score=True)
cv_dictKfold_l1 = cross_val_score(lr_l1,CountVec_train_real,y_train,cv = kfolds, scoring ='accuracy')
print('The cross validation result for logistics regression with l1 regulation is ', cv_dict_l1)
       

The cross validation result for logistics regression without regulation is  {'fit_time': array([1.15430307, 0.98953795, 1.06057692, 1.00339603, 1.00738502]), 'score_time': array([0.02622986, 0.02428079, 0.02401519, 0.02459788, 0.02545309]), 'test_score': array([0.72795497, 0.74953096, 0.74765478, 0.73827392, 0.74084507]), 'train_score': array([0.98874032, 0.9859254 , 0.98522167, 0.98615998, 0.98827392])}
The cross validation result for logistics regression with l2 regulation is  {'fit_time': array([0.68969083, 0.72944999, 0.72613811, 0.68315196, 0.67498088]), 'score_time': array([0.0267489 , 0.02406406, 0.02332592, 0.02389216, 0.02323103]), 'test_score': array([0.77016886, 0.7945591 , 0.79549719, 0.78705441, 0.78028169]), 'train_score': array([0.93760263, 0.93126906, 0.93361483, 0.93079991, 0.93269231])}
The cross validation result for logistics regression with l1 regulation is  {'fit_time': array([0.30860209, 0.29218197, 0.29550123, 0.2941072 , 0.29299712]), 'score_time': array([0.023

In [199]:
#check for accuracy using cross_val_score KFolds for model without regulation 
cv_dictKfold.mean()

0.7350371707881336

In [201]:
#check for accuracy using cross_val_score KFolds for model with l1 regulation 
cv_dictKfold_l1.mean()

0.7785668439391161

In [202]:
#check for accuracy using cross_val_score KFolds for model with l2 regulation 
cv_dictKfold_l2.mean()

0.7862612676155679

<span style="color:blue"> 
By comparing the logistric regression with regulation, l1 regulation and l2 regulation using cross_validation methods, we can see that the model without regulation has a train score very close to 1, which is an indication of overfitting. Both l2 and l1 methods yield a normal train score, but l2 has a better accuracy than l1. 


* Bernoulli Naive Bayes.Implement a Bernoulli Naive Bayes classifier to predict the probability of whether each tweet is about a real disaster. Train this classifier on the training set, and report its F 1-score on the development set.
    
    Important: For this question you should implement the classifier yourself similar to what was shown in class, without using any existing machine learning libraries such as sklearn. You may only use basic libraries such as numpy.
    
    As you work on this problem, you may find that some words in the vocabulary occur in the development set but are not in the training set. As a result, the standard Naive Bayes model learns to assign them an occurrence probability of zero, which becomes problematic when we observe this "zero probability" event on our development set.
    
    The solution to this problem is a form of regularization called Laplace smoothing or additive smoothing. The idea is to use "pseudo-counts", i.e. to increment the number of times we have seen each word or document by some number of "virtual" occurrences α. Thus, the Naive Bayes model will behave as if every word or document has been seen at least α times.
    
    More formally, the ψjk parameter of Bernoulli Naive Bayes is the probability of observing word j within class k. Its normal maximum likelihood estimate is
    
    $$ \frac{\phi_k}{\sum_l \phi_l} = \frac{n_jk}{n_k}$$
    
    


where nk is the number of documents of class k and n j k is the number of documents of class k that contain word j . In Laplace smoothing, we increment each counter n j k by α (thus we count each word an extra α times), and the resulting estimate for ψ j k becomes:

$$ \frac{\phi_k}{\sum_l \phi_l} = \frac{n_k + \alpha}{n_k+2\alpha}$$

In [207]:
#Retrieve below code from course lecture slides#6
n = CountVec_train_real.shape[0] # size of the dataset
d = CountVec_train_real.shape[1] # number of features in our dataset
K = 2 # number of clases

# these are the shapes of the parameters
psis = np.zeros([K,d])
phis = np.zeros([K])

# we now compute the parameters
for k in range(K):
    vecTrain_k = CountVec_train_real[y_train == k]
    psis[k] = np.mean(vecTrain_k, axis=0)
    phis[k] = vecTrain_k.shape[0] / float(n)

# print out the class proportions
print(phis)

[0.56370801 0.43629199]


In [214]:
def nb_predictions(x, psis, phis):
    #This returns class assignments and scores under the NB model.
    
    #We compute \arg\max_y p(y|x) as \arg\max_y p(x|y)p(y)

    # adjust shapes
    n, d = x.shape
    x = np.reshape(x, (1, n, d))
    psis = np.reshape(psis, (K, 1, d))
    
    # clip probabilities to avoid log(0)
    psis = psis.clip(1e-14, 1-1e-14)
    
    # compute log-probabilities
    logpy = np.log(phis).reshape([K,1])
    logpxy = x * np.log(psis) + (1-x) * np.log(1-psis)
    logpyx = logpxy.sum(axis=2) + logpy

    return logpyx.argmax(axis=0).flatten(), logpyx.reshape([K,n])

idx, logpyx = nb_predictions(CountVec_train_real, psis, phis)
idx_valid, logpyx_valid = nb_predictions(CountVec_valid_real, psis, phis)
print(idx[:10])

[0 0 0 0 1 0 1 1 1 0]


In [217]:
print('Mean of predicted results on training data: %.4f' % (idx==y_train).mean())
print('Mean of predicted results on valid data: %.4f' % (idx_valid==y_valid).mean())
print("F1 score on the training set: %.4f" % f1_score(y_train, idx))
print("F1 score on the dev set: %.4f" % f1_score(y_valid, idx_valid))


Mean of predicted results on training data: 0.8882
Mean of predicted results on valid data: 0.7916
F1 score on the training set: 0.8645
F1 score on the dev set: 0.7344


* Model comparison。 You just implemented a generative classifier and a discriminative classifier. Reflect on the following:
    * Which model performed the best in predicting whether a tweet is of a real disaster or not? Include your performance metric in your response. Comment on the pros and cons of using generative vs discriminative models.
    * Think about the assumptions that Naive Bayes makes. How are the assumptions differ- ent from logistic regressions? Discuss whether it’s valid and efficient to use Bernoulli Naive Bayes classifier for natural language texts.

<span style="color:blue"> 
The bernouli naives bayes performes the best in predicting whether a teweet is of a real disaster as it has a F1_sore of 0.86 which is higher than all the logistics regression models (with or without regularization). The fundamental difference between discriminative models and generative models is that Discriminative models learn the (hard or soft) boundary between classes and Generative models model the distribution of individual classes. Naive Bayes (generative classifiers)  assume some functional form for $P(Y)$, $P(X|Y)$, estimate parameters of $P(X|Y)$, $P(Y)$ directly from training data, and use Bayes rule to calculate $P(Y|X)$. While Logistics regression (Discriminative Classifiers) assume some functional form for $P(Y|X)$ and estimates parameters of $P(Y|X)$ directly from training data. 
    
The advantage of using generative classifiers is that we can assume the occurence of one word is not impacted by the occurence of the other word. So if the assumption is true, it will give us better accuracy. 
    
The discriminative classifiers may not be very suitable in our natural language processing cases as some words will almost always appear together, which may violate the conditional independcy in this situation. 

* N-gram model.The N-gram model is similar to the bag of words model,but instead of using individual words we use N-grams, which are contiguous sequences of words. For example, using N = 2, we would says that the text “Alice fell down the rabbit hole” consists of the se- quence of 2-grams: ["Alice fell", "fell down", "down the", "the rabbit", "rabbit hole"], and the following sequence of 1-grams: ["Alice", "fell", "down", "the", "rabbit", "hole"]. All eleven of these symbols may be included in the vocabulary, and the feature vector x is defined according to xi = 1 if the i’th vocabulary symbol occurs in the tweet, and xi = 0 otherwise. Using N = 2, construct feature representations of the tweets in the training and development tweets. Again, you should choose a threshold M , and only include symbols in the vocabulary that occur in at least M different tweets in the training set. Discuss how you chose the thresh- old M, and report the total number of 1-grams and 2-grams in your vocabulary. In addition, take 10 2-grams from your vocabulary, and print them out.


    Then, implement a logistic regression and a Bernoulli classifier to train on 2-grams. You may reuse the code in (e) and (f). You may also choose to use or not use a regularization term, depending on what you got from (e). Report your  results on training and development set. Do these results differ significantly from those using the bag of words model? Discuss what this implies about the task.
    
    Again, we suggest using CountVectorizer to construct these features. In order to include both 1-gram and 2-gram features, you can set ngram_range=(1,2). Note also that in this case, since there are probably many different 2-grams in the dataset, it is especially important carefully set min_df in order to avoid run-time and memory issues.


## Logistics Regression + N-Gram

In [242]:
#ngram for log regression
for i in range(1,9):
    CountVec_ngram = CountVectorizer(ngram_range = (1, 2), min_df = i)
    #transform
    Countgram_train = CountVec_ngram.fit_transform(x_train).toarray()
    Countgram_valid = CountVec_ngram.transform(x_valid).toarray()
    logreg_gram = LogisticRegression()
    logreg_gram.fit(Countgram_train,y_train)
    predict_y_gram = logreg_gram.predict(Countgram_valid)
    print('With M = %s, F1 score = %.4f' % (i, f1_score(y_valid, predict_y_gram)))
    
 
#use M = 3
count_gram_real = CountVectorizer(ngram_range = (1, 2),binary=True, min_df=3)
Countgram_train_real = count_gram_real.fit_transform(x_train).toarray()
gramnames = count_gram_real.get_feature_names()
print(gramnames)
Countgram_valid_real = count_gram_real.transform(x_valid).toarray()
Countgram_valid_real.shape


With M = 1, F1 score = 0.7486
With M = 2, F1 score = 0.7450
With M = 3, F1 score = 0.7451
With M = 4, F1 score = 0.7347
With M = 5, F1 score = 0.7374
With M = 6, F1 score = 0.7388
With M = 7, F1 score = 0.7349
With M = 8, F1 score = 0.7358
['0104', '010401', '010401 utc20150805', '02', '05', '06', '0day', '0day bug', '10', '100', '1000', '101', '101 to', '1030', '1030 pm', '1030pm', '10th', '11', '11yearold', '11yearold boy', '12', '1200', '12000', '12000 nigerian', '125', '13', '13 reason', '13000', '133', '14', '14 lez', '15', '15 km', '15 saudi', '150', '150401', '150401 1000', '16', '1620', '16yr', '16yr old', '17', '18', '18w', '18w cree', '18wheeler', '19', '19 km', '1980', '1980 prebreak', '1st', '20', '20 amp', '2005', '2010', '2011', '2013', '2013 the', '2014', '2015', '2015 australia', '2015 prebreak', '2030', '21', '22', '23', '24', '25', '25 home', '25 kill', '26', '27', '29', '2nd', '2pcs', '2pcs 18w', '30', '30 fire', '300', '300000', '300w', '300w curve', '31', '31 md', 

(2284, 4815)

In [235]:
#find out f1_score and the 10 most common names to determine if the tweet is about disaster
lr_l2_gram = LogisticRegression(penalty = 'l2')
lr_l2_gram.fit (Countgram_train_real, y_train)
prediction_y_gram = lr_l2_gram.predict(Countgram_valid_real)
print('F1 score with logreg: %.4f' % f1_score(y_valid, prediction_y_gram))
weights = lr_l2_gram.coef_[0]
sorted_weight_index = np.argsort(np.abs(lr_l2_gram.coef_[0])) #return the index of the sorted weights 
for i in sorted_weight_index[-10:]:
    print(gramnames[i])

F1 score with logreg: 0.7492
drought
terrorism
storm
evacuate
typhoon
earthquake
wildfire
bombing
massacre
hiroshima


In [236]:
## Count # of 1-gram and 2gram vocab

twogram = []
for words in gramnames:
    if " " in words: 
        twogram.append(words)
twogramNum = len(twogram)   
onegramNum = len(gramnames)-twogramNum
        
print('The number of 2gram names are: ', twogramNum)
print('The number of 1gram names are: ', onegramNum)

The number of 2gram names are:  1877
The number of 1gram names are:  2938


### Naive Bayes + N-Gram

In [240]:

n_gram = Countgram_train_real.shape[0] # size of the dataset
d_gram = Countgram_train_real.shape[1] # number of features in our dataset
K_gram = 2 # number of clases

# these are the shapes of the parameters
psis = np.zeros([K_gram,d_gram])
phis = np.zeros([K_gram])

# we now compute the parameters
for k in range(K_gram):
    gramTrain_k = Countgram_train_real[y_train == k]
    psis[k] = np.mean(gramTrain_k, axis=0)
    phis[k] = gramTrain_k.shape[0] / float(n)

# print out the class proportions
print(phis)

[0.56370801 0.43629199]


In [241]:
def nb_predictions_gram(x, psis, phis):
    #This returns class assignments and scores under the NB model.
    
    #We compute \arg\max_y p(y|x) as \arg\max_y p(x|y)p(y)

    # adjust shapes
    n_gram, d_gram = x.shape
    x = np.reshape(x, (1, n_gram, d_gram))
    psis = np.reshape(psis, (K_gram, 1, d_gram))
    
    # clip probabilities to avoid log(0)
    psis = psis.clip(1e-14, 1-1e-14)
    
    # compute log-probabilities
    logpy = np.log(phis).reshape([K_gram,1])
    logpxy = x * np.log(psis) + (1-x) * np.log(1-psis)
    logpyx = logpxy.sum(axis=2) + logpy

    return logpyx.argmax(axis=0).flatten(), logpyx.reshape([K_gram,n_gram])

idx_gram, logpyx_gram = nb_predictions(Countgram_train_real, psis, phis)
idx_valid_gram, logpyx_valid_gram = nb_predictions(Countgram_valid_real, psis, phis)
print(idx[:10])

print('Mean of predicted results on training data using Naive Bayes and N gram: %.4f' % (idx_gram==y_train).mean())
print('Mean of predicted results on valid data: %.4f' % (idx_valid_gram==y_valid).mean())
print("F1 score on the training set: %.4f" % f1_score(y_train, idx_gram))
print("F1 score on the dev set: %.4f" % f1_score(y_valid, idx_valid_gram))

[0 0 0 0 1 0 1 1 1 0]
Mean of predicted results on training data using Naive Bayes and N gram: 0.8698
Mean of predicted results on valid data: 0.8021
F1 score on the training set: 0.8352
F1 score on the dev set: 0.7303


<span style="color:blue"> Based on the result comparison, the 10 most common words that indicating the disaster tweet change slightly between the N-gram and bag of words methods, but overall the predicted results and F1 score doesn't differ a lot between both models. This may implies that there are many ways for us to do this problem, different models may yeild an overall similar result and lead to the same direction. 

* Determine performance with the test set Re-build your feature vectors and re-train your preferred classifier (either bag of word or n-gram using either logistic regression or Bernoulli naive bayes) using the entire Kaggle training data (i.e. using all of the data in both the training and development sets). Then, test it on the Kaggle test data. Submit your results to Kaggle, and report the resulting F 1-score on the test data, as reported by Kaggle. Was this lower or higher than you expected? Discuss why it might be lower or higher than your expectation.

In [252]:
testData.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [256]:
newData.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [266]:
## Preprocessing test data -- Lowercase----------------------------------------------------------------------------------------------------------
testData['text'] = testData['text'].str.lower()



## strip urls-------------------------------------------------------------------------------------------------------------------
for l in range(0,len(testData['text'])): 
    testData['text'][l] = re.sub(r'http\S+', '', testData['text'][l])
    testData['text'][l] = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', testData['text'][l])

    
## get ride of punctuation----------------------------------------------------------------------------------------------------------
testData['text'] = testData['text'].str.translate(str.maketrans('', '', string.punctuation))



## strip stop words ----------------------------------------------------------------------------------------------------------
text_token_test = []
temp_test = testData['text']
for i in range(0,len(testData['text'])): 
    text_token_test.append(word_tokenize(temp_test[i]))
for row in range(0,len(text_token_test)):
    for word in text_token_test[row]:
        if word in stopwords.words('english'):
            text_token_test[row].remove(word)
        
            
    text_token_test[row] =  (" ").join(text_token_test[row])   

In [267]:
## Lamatize Test data----------------------------------------------------------------------------------------------------------------

nlp = spacy.load('en_core_web_sm')

clean_token_test = []
lemmatized_sentence_test = []

for l in range(0,len(text_token_test)):
    text_token_test[l] = nlp(text_token_test[l])
    for tokes in text_token_test[l]:
        clean_token_test.append(tokes)
        
    lemmatized_sentence_test.append(" ".join([tokes.lemma_ for tokes in text_token_test[l]]))  
    

clean_sent_test = pd.DataFrame({'clean_Text':lemmatized_sentence_test})
new_testData = pd.concat([testData, clean_sent_test],join = 'outer', axis = 1)
   

In [282]:
new_testData.head()

,id,keyword,location,text,clean_Text
0,0,NaN,NaN,just happened a terrible car crash,happen terrible car crash
1,2,NaN,NaN,heard about earthquake is different cities sta...,hear earthquake different city stay safe everyone
2,3,NaN,NaN,there is a forest fire at spot pond geese are ...,be forest fire spot pond geese flee across str...
3,9,NaN,NaN,apocalypse lighting spokane wildfires,apocalypse lighting spokane wildfire
4,11,NaN,NaN,typhoon soudelor kills 28 in china and taiwan,typhoon soudelor kill 28 china taiwan


In [275]:
## use M = 2
count_vect_final = CountVectorizer(binary=True, min_df=2)
CountVec_train_final = count_vect_final.fit_transform(new_trainData['clean_Text']).toarray()
vnames = count_vect_real.get_feature_names()
#print(vnames)
CountVec_train_final.shape

(7613, 5562)

In [ ]:
#-------------------------------next page -----------------------------------------

In [ ]:
#------------------------------------------------------------------------------------

In [274]:
# convert dev set to bag of word
count_test_final = count_vect_final.transform(new_testData['clean_Text']).toarray()
# sanity check
count_test_final.shape

(3263, 5562)

In [283]:
lr_final = LogisticRegression(penalty = 'l2')
lr_final.fit(CountVec_train_final, new_trainData['target'])
predict_test_final = lr_final.predict(count_test_final)


In [279]:
import csv
with open('Final result.csv', mode='w+') as rfile:
    rfile.write('id,target\n')
    for (id, tg) in zip(new_testData['id'].values, predict_test_final):
        rfile.write('%s,%s\n'%(id, tg))

<span style="color:blue"> Based on the F1 score return from Kaggle, we get 0.78608 F1 score for this problem. This score is within our expectation as we chosed Bag of words and Logistics regressions as our methods, which is pretty aligned with the F1 score we received during the training and validation process. I expect our F1 score to be higher if we use naive bayes method. 